## Example Binary Classification using proposed AutoML API
### Disclaimer: This is for demo and private preview purposes only.
### Disclaimer: This notebook uses features that may be broken or unsupported at any time.

In [ ]:
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json" 
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json"
#i "nuget:https://mlnetcli.blob.core.windows.net/mlnetcli/index.json"

// add nightly build for ml.net
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json"

Restore sources https://mlnetcli.blob.core.windows.net/mlnetcli/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json

In [ ]:
#r "nuget:MLNetAutoML.InteractiveExtension,0.2.0"

// XPlot is used to plot trials during training because it's strong-named. However it's recommended to use Ploty.Net to plot digrams in notebook.
#r "nuget:Microsoft.DotNet.Interactive.Formatting, 1.0.0-beta.22256.1"
#r "nuget:XPlot.Plotly.Interactive,4.0.6"
#r "nuget:Microsoft.ML.AutoML,0.20.0-preview.22317.1"
#r "nuget:Microsoft.Data.Analysis,0.20.0-preview.22317.1"

Restore sources https://mlnetcli.blob.core.windows.net/mlnetcli/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json Installed Packages Microsoft.Data.Analysis, 0.20.0-preview.22317.1 Microsoft.DotNet.Interactive.Formatting, 1.0.0-beta.22256.1 Microsoft.ML.AutoML, 0.20.0-preview.22317.1 MLNetAutoML.InteractiveExtension, 0.2.0 XPlot.Plotly.Interactive, 4.0.6

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

Loading extensions from `XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

Loading extensions from `MLNetAutoML.InteractiveExtension.dll`

In [ ]:
// Import usings.
using Microsoft.Data.Analysis;
using System;
using System.IO;
using Microsoft.ML;
using Microsoft.ML.AutoML;
using Microsoft.ML.Data;
using MLNetAutoML.InteractiveExtension;

In [ ]:
//Load File
var trainDataPath = Path.Combine(Directory.GetCurrentDirectory(), "data", "titanic.csv");
var df = DataFrame.LoadCsv(trainDataPath);

//Setup AutoML Pipeline
var context = new MLContext();

var pipeline = context.Auto().Featurizer(df, excludeColumns: new[]{"Survived"})
                        .Append(context.Transforms.Conversion.ConvertType("Survived", "Survived", DataKind.Boolean))
						.Append(context.Auto().BinaryClassification(labelColumnName: "Survived", useFastForest: false));

// Configure AutoML
var trainTestSplit = context.Data.TrainTestSplit(df, 0.1);
var monitor = new NotebookMonitor();

var experiment = context.Auto().CreateExperiment()
                    .SetPipeline(pipeline)
                    .SetTrainingTimeInSeconds(50)
                    .SetDataset(trainTestSplit.TrainSet, trainTestSplit.TestSet)
                    .SetEvaluateMetric(BinaryClassificationMetric.Accuracy, "Survived", "PredictedLabel")
                    .SetMonitor(monitor);

// Configure Visualizer			
monitor.SetUpdate(monitor.Display());

// Start Experiment
var res = await experiment.RunAsync();

index,Trial,Metric,Trainer,Parameters
0,0,0.5897436,ReplaceMissingValues=>OneHotEncoding=>FeaturizeText=>Concatenate=>SdcaLogisticRegressionBinary,"{""0"":{""OutputColumnNames"":[""PassengerId"",""Pclass"",""Age"",""SibSp"",""Parch"",""Fare""],""InputColumnNames"":[""PassengerId"",""Pclass"",""Age"",""SibSp"",""Parch"",""Fare""]},""1"":{""OutputColumnNames"":[""Sex"",""Cabin"",""Embarked""],""InputColumnNames"":[""Sex"",""Cabin"",""Embarked""]},""2"":{""InputColumnName"":""Name"",""OutputColumnName"":""Name""},""3"":{""InputColumnNames"":[""Name"",""PassengerId"",""Pclass"",""Age"",""SibSp"",""Parch"",""Fare"",""Sex"",""Cabin"",""Embarked""],""OutputColumnName"":""Features""},""4"":{},""5"":{""L1Regularization"":1,""L2Regularization"":0.1,""LabelColumnName"":""Survived"",""FeatureColumnName"":""Features""}}"
1,1,0.82051283,ReplaceMissingValues=>OneHotHashEncoding=>FeaturizeText=>Concatenate=>LbfgsLogisticRegressionBinary,"{""0"":{""OutputColumnNames"":[""PassengerId"",""Pclass"",""Age"",""SibSp"",""Parch"",""Fare""],""InputColumnNames"":[""PassengerId"",""Pclass"",""Age"",""SibSp"",""Parch"",""Fare""]},""1"":{""OutputColumnNames"":[""Sex"",""Cabin"",""Embarked""],""InputColumnNames"":[""Sex"",""Cabin"",""Embarked""]},""2"":{""InputColumnName"":""Name"",""OutputColumnName"":""Name""},""3"":{""InputColumnNames"":[""Name"",""PassengerId"",""Pclass"",""Age"",""SibSp"",""Parch"",""Fare"",""Sex"",""Cabin"",""Embarked""],""OutputColumnName"":""Features""},""4"":{},""5"":{""L1Regularization"":1,""L2Regularization"":1,""LabelColumnName"":""Survived"",""FeatureColumnName"":""Features""}}"
2,2,0.5897436,ReplaceMissingValues=>OneHotHashEncoding=>FeaturizeText=>Concatenate=>SdcaLogisticRegressionBinary,"{""0"":{""OutputColumnNames"":[""PassengerId"",""Pclass"",""Age"",""SibSp"",""Parch"",""Fare""],""InputColumnNames"":[""PassengerId"",""Pclass"",""Age"",""SibSp"",""Parch"",""Fare""]},""1"":{""OutputColumnNames"":[""Sex"",""Cabin"",""Embarked""],""InputColumnNames"":[""Sex"",""Cabin"",""Embarked""]},""2"":{""InputColumnName"":""Name"",""OutputColumnName"":""Name""},""3"":{""InputColumnNames"":[""Name"",""PassengerId"",""Pclass"",""Age"",""SibSp"",""Parch"",""Fare"",""Sex"",""Cabin"",""Embarked""],""OutputColumnName"":""Features""},""4"":{},""5"":{""L1Regularization"":1,""L2Regularization"":0.1,""LabelColumnName"":""Survived"",""FeatureColumnName"":""Features""}}"
3,3,0.82051283,ReplaceMissingValues=>OneHotHashEncoding=>FeaturizeText=>Concatenate=>FastTreeBinary,"{""0"":{""OutputColumnNames"":[""PassengerId"",""Pclass"",""Age"",""SibSp"",""Parch"",""Fare""],""InputColumnNames"":[""PassengerId"",""Pclass"",""Age"",""SibSp"",""Parch"",""Fare""]},""1"":{""OutputColumnNames"":[""Sex"",""Cabin"",""Embarked""],""InputColumnNames"":[""Sex"",""Cabin"",""Embarked""]},""2"":{""InputColumnName"":""Name"",""OutputColumnName"":""Name""},""3"":{""InputColumnNames"":[""Name"",""PassengerId"",""Pclass"",""Age"",""SibSp"",""Parch"",""Fare"",""Sex"",""Cabin"",""Embarked""],""OutputColumnName"":""Features""},""4"":{},""5"":{""NumberOfLeaves"":4,""MinimumExampleCountPerLeaf"":20,""NumberOfTrees"":4,""MaximumBinCountPerFeature"":255,""FeatureFraction"":1,""LearningRate"":0.09999999999999998,""LabelColumnName"":""Survived"",""FeatureColumnName"":""Features""}}"
4,4,0.82051283,ReplaceMissingValues=>OneHotEncoding=>FeaturizeText=>Concatenate=>LbfgsLogisticRegressionBinary,"{""0"":{""OutputColumnNames"":[""PassengerId"",""Pclass"",""Age"",""SibSp"",""Parch"",""Fare""],""InputColumnNames"":[""PassengerId"",""Pclass"",""Age"",""SibSp"",""Parch"",""Fare""]},""1"":{""OutputColumnNames"":[""Sex"",""Cabin"",""Embarked""],""InputColumnNames"":[""Sex"",""Cabin"",""Embarked""]},""2"":{""InputColumnName"":""Name"",""OutputColumnName"":""Name""},""3"":{""InputColumnNames"":[""Name"",""PassengerId"",""Pclass"",""Age"",""SibSp"",""Parch"",""Fare"",""Sex"",""Cabin"",""Embarked""],""OutputColumnName"":""Features""},""4"":{},""5"":{""L1Regularization"":1,""L2Regularization"":1,""La